# Combinar Datasets: Concat y Append

Algunos de los estudios más interesantes de datos vienen de combinar diferentes fuentes de datos. Estas operaciones pueden involucrar de todo, desde la concatenación de dos datasets diferentes, hasta cosas más complicadas como hacer cruces y uniones de bases de datos que tengan en cuenta solapes de información.

Las ``Series`` y los ``DataFrames`` están construídos con este tipo de operaciones en mente, y Pandas incluye diversas funciones y métodos que hacer este tipo de Data Wrangling (manejo de datos) rápido y entendible.

Vamos a echarle un vistazo a una simple concatenación de una ``Serie`` y un ``DataFrame`` con el comando pd.concat, luego nos sumergiremos en operaciones más sofisticadas con cruces y uniones en memoria en Pandas.

Comencemos con los imports necesarios:

In [2]:
import pandas as pd
import numpy as np

Para ahorrarnos algo de tiempo, vamos a definir esta función que crea automáticamente un ``DataFrame`` de una determinada manera:

In [3]:
def fabricador_df(cols, ind):
    """Super constructor de Dataframes!"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# ejemplo de DataFrame
fabricador_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


Además, vamos a crear una clase sencilla que nos permita mostrar varios ``DataFrames`` uno al lado del otro. El código hace uso de un método llamado ``_reprhtml_`` especial, que IPython usa para implementar su potente representación de objetos:

In [4]:
class display(object):
    """Representador HTML de múltiples objetos"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

Tranquilos, el uso de esto va a ser interesante para la siguiente sección. :)

## Vamos a repasar: Concatenación de Arrays de Numpy

La concatenación de objetos de tipo ``Series y DataFrames`` es muy similar a la concatenación de Numpy Arrays, que ya sabemos que se puede hacer con la función ``np.concatenate`` como vimos en el primer capítulo de Pandas 1.

Recordemos, podemos combinar el contenido de dos o más arrays en uno solo:

In [5]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

El primer argumento es una lista o una tupla de Arrays para concatenar. Adicionalmente, requiere de un argumento adicional que nos permite especificar que eje o ``Axis`` queremos combinar:

In [6]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=0)

array([[1, 2],
       [3, 4],
       [1, 2],
       [3, 4]])

## Concatenación sencilla con ``pd.concat``

Pandas tiene la función ``pd.concat()`` que nos permite, con una sintaxis similar a ``np.concatenate()`` hacer lo mismo, pero con un número de opciones superior, más adelante las comentaremos:

```python
# Basado en la versión v0.18 de Pandas
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

``pd.concat()`` puede ser usado para unir/concatenar objetos de tipo ``Series`` or ``DataFrame``, como hace ``np.concatenate()`` se puede usar para concatenar simples Arrays:

In [7]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
print(ser1)
ser2 = pd.Series(['D', 'E', 'F'], index=[1, 2, 3])
print(ser2)
pd.concat([ser1, ser2], axis = 0).reset_index(drop=True)

1    A
2    B
3    C
dtype: object
1    D
2    E
3    F
dtype: object


0    A
1    B
2    C
3    D
4    E
5    F
dtype: object

Como podíamos esperar, sirve para concatenar objetos de Alta-Dimensionalidad como nuestro conocido ``DataFrame``

In [8]:
df1 = fabricador_df('AB', [1, 2])
df2 = fabricador_df('AB', [1, 2])

display('df1','df2','pd.concat([df1, df2], axis=1)')

df1
    A   B
1  A1  B1
2  A2  B2

df2
    A   B
1  A1  B1
2  A2  B2

pd.concat([df1, df2], axis=1)
    A   B   A   B
1  A1  B1  A1  B1
2  A2  B2  A2  B2

Por defecto, la concatenación se hace a nivel fila entre el ``DataFrame`` (por ejemplo, con un ``axis=0``). Como ocurre con ``np.concatenate``, ``pd.concat`` también permite la especificación de un eje o axis por el cual vamos a hacer la concatenación. Vamos a ver este ejemplo:

In [9]:
df3 = fabricador_df('AB', [0, 1])
df4 = fabricador_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis=0)")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis=0)
     A    B    C    D
0   A0   B0  NaN  NaN
1   A1   B1  NaN  NaN
0  NaN  NaN   C0   D0
1  NaN  NaN   C1   D1

Si te fijas, podemos usar el equivalente ya conocido de ``axis = 1``. En esencia le estamos diciendo a Pandas que la concatenación tiene que ser a nivel ``Columna``.

### Índices duplicados

Una diferencia ``SUPER IMPORTANTE`` entre los dos métodos de las librerías de Numpy y Pandas es que la concatenación en Pandas mantiene los índices, ``íncluso si tenemos índices duplicados en el resultado``. Vamos a ver un ejemplo:

In [10]:
x = fabricador_df('AB', [0, 1])
y = fabricador_df('AB', [2, 3])
y.index = x.index  # Hace índices duplicados! :-O
display('x', 'y', 'pd.concat([x, y], axis = 0)') # Siempre debemos resetar los índices para evitar el caos, ahora vemos cómo :-)

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


Date cuenta que se nos han repetido los índices. Que esto ocurra es válido, este resultado puede ser interesante en ciertas ocasiones. ``pd.concat()`` nos da algunas alternativas.

``¿Se te ocurre alguna?``.


#### Capturar las duplicidades como un error:

Si quieres que simplemente se verifique la duplicidad de los índices en el resultado de un ``pd.concat()``, podemos especidicar la marca o flac ``verify_integrity``. Con este parámetro a True, la concatenación lanzará una excepción si tenemos índices duplicados. Aquí va un ejemplo, dónde añadimos un print para capturar el mensaje de error en nuestro log:

In [11]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


#### Ignorar el índice:

A veces el índice no es del todo necesario y lo mejor es simplemente ignorarlo. Para esta opción podemos usar el parámetro ``ignore_filter``. Cuando se introduce igualado a True, la concatenación se creará con un nuevo entero incremental resultante de la concatenación:

In [12]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


#### Añadir múltiples índices:

Otra opción es usar el parámetro de ``keys``, que nos ayuda a especificar una etiqueta para cada una de las fuentes; el resultado se representará con un doble índice jerárquico:

In [13]:
display('x', 'y', "pd.concat([x, y], keys=['VISITAS', 'ANALITICAS'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['VISITAS', 'ANALITICAS'])
               A   B
VISITAS    0  A0  B0
           1  A1  B1
ANALITICAS 0  A2  B2
           1  A3  B3

El resultado será un ``DataFrame indexado``, concepto que ya mostramos en el apartado de Pandas 1, Indexación jerárquica para transformar los datos en la representación gráfica en la que estamos interesados.

### Concatenación con Joins

En los ejemplos que hemos visto, nos hemos centrado en la concatenación de ``DataFrames`` con nombres columnas iguales. En la práctica, los datos que provienen de diferentes fuentes suelen tener diferentes sets de nombres de columnas, en este caso pd.concat ofrece últiles opciones. Consideremos la concatenación de los siguientes ``DataFrames``, como podéis ver, hemos alterado los nombres de columnas:

In [14]:
df5 = fabricador_df('ABC', [1, 2])
df6 = fabricador_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

Por defecto, las combinaciones en las que no hay datos se rellenan con NANs. Para cambiar esto, tenemos que especificar uno de los múltiples inputs que nos ofrecen los parámetros de ``join`` y ``join_axes``. Por defecto, el join es una unión de las columnas input (``join = 'outer'``), pero podemos cambiarlo a una simple intersección, con el valor del parámetro del método ``join='inner'``:

In [15]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

La combinación de las opciones de la función ``pd.concat`` nos permite un amplio abanico de posibles tipos de cruce entre datasets. Tenerlo en cuenta para usarlas en los momentos clave.

### El método ``append()``

Debido a que la concatenación directa de arrays es muy común, Para las ``Series`` y los ``DataFrame`` tenemos también el método ``append`` para poder hacer lo mismo, pero con menos pasos. Por ejemplo, en vez de tener que llamar ``pd.concat([df1, df2])``, podemos usar ``df1.append(df2)``:

In [16]:
display('df1', 'df2', 'df1.append(df2)')

,A,B
1,A1,B1
2,A2,B2
,A,B
1,A1,B1
2,A2,B2
,A,B
1,A1,B1
2,A2,B2
1,A1,B1
2,A2,B2


! CUIDADO!!!! Este método ya no pertenece a Pandas 2.0, para esto tenemos contat :)

Ten muy en cuenta que en los métodos append() y extend() de las listas de Python modifican al propio constructor, ``cosa que no ocurre con este método.`` En el caso de Pandas no se modifica el objeto original, se crea uno nuevo con los datos combinados. Es además un método mucho mñas eficiente, ya que involucra la creación de un nuevo índice y además de un espacio nuevo en la memoria.

# Combinar Datasets: Merge and Join

Una característica clave de Pandas es sus operaciones de cruce y unión de alto rendimiento en memoria. Si alguna vez has trabajado con bases de datos, te resultará muy familiar la necesidad de pensar en el consumo de memoria y el tiempo de ejecución en este tipo de operaciones. La mejor manera de comenzar con los cruces es presentando la función de ``pd.merge``. Vamos a ver varios ejemplos de uso.

Cómo siempre vamos a hacer uso de nuestra clase ``display()``

In [17]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Algebra Relacional

El comportamiento implementado en ``pd.merge`` es una parte de lo que se conoce como algebra relacional, que es un conjunto de reglas formales para manipular datos que son relacionales. Sus bases están presentes en la mayoría de bases de datos relaciones.

La fortaleza del algebra relacional es la manera de procesar conjuntamente un conjunto de operaciones *primitivas*, las cuales son la base de un conjunto de operaciones más complejas que se pueden aplicar a cualquier dataset. Con este conjunto de conceptos podremos abordar la totalidad de operaciones.

Pandas implementa muchas de estas operaciones *primitivas* o de base en la función ``pd.merge`` y en la de ``pd.join()``.

## Categorías de Joins

La función ``pd.merge()`` implementa diversos tipos de joins: Los conocidos en inglés como, *one-to-one*, *many-to-one*, and *many-to-many*. Estos tres tipos de cruces son llamados/procesados con la misma función/interfaz de ``pd.merge()``, ahora vamos a ver cómo y cuándo usar cada uno.

### One-to-one joins

Quizás el más simple de todas es la unión uno a uno o one-to-one join, que es muy similar a la que hemos visto sobre las concatenaciones orientadas a columnas en el capítulo anterior. 

Como ejemplo, vamos a tomar dos ``DataFrames`` que contienen información de empleados de una compañia:


In [18]:
df1 = pd.DataFrame({'Empleado': ['Jose', 'Jesus', 'Gema', 'Julia'],
                    'Departamento': ['Contabilidad', 'Ingenieria', 'Ingenieria', 'RRHH']})
df2 = pd.DataFrame({'Empleado': ['Gema', 'Jose', 'Jesus', 'Julia'],
                    'Fecha_contratacion': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,Empleado,Departamento
0,Jose,Contabilidad
1,Jesus,Ingenieria
2,Gema,Ingenieria
3,Julia,RRHH
,Empleado,Fecha_contratacion
0,Gema,2004
1,Jose,2008
2,Jesus,2012
3,Julia,2014


Para combinar esta información en un solo ``DataFrame``, podemos usar la función ``pd.merge()`` de esta manera:

In [19]:
df3 = pd.merge(df1, df2)
df3

,Empleado,Departamento,Fecha_contratacion
0,Jose,Contabilidad,2008
1,Jesus,Ingenieria,2012
2,Gema,Ingenieria,2004
3,Julia,RRHH,2014


In [20]:
type(df3)

pandas.core.frame.DataFrame

Esta función ``pd.merge()`` reconoce que cada ``DataFrame`` tiene una columna empleado, y automáticamente une estas columnas usando esta clave común. El resultado de estea unión es un nuevo ``Dataframe`` que combina la información de estos dos inputs.

Acuérdate que el orden de las entradas para cada columna no tiene que mantenerse, en este caso el orden de la columna empleado difiere entre ambos ``DataFrames`` y la función ``pd.merge()`` trabaja con esto correctamente.

Adicionalmente, ten en cuenta que la unión por lo general descarta/no usa los índices, a excepción del caso especial de los merges/uniones por índice. (Ver para más detalle las secciones de ``left_index`` y ``right_index``).

### Many-to-one joins

Many-to-one joins o las uniones de Muchos-a-Uno son las que una de las dos columnas de clave contiene elementos duplicados. Para estos casos, el ``DataFrame`` resultante preservará los duplicados. Vamos a aplicar a nuestro ejemplo RRHH Database:

In [21]:
df4 = pd.DataFrame({'Departamento': ['Contabilidad', 'Ingenieria', 'RRHH',],
                    'supervisor': ['Javier', 'Daniela', 'Carlota']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  Empleado  Departamento  Fecha_contratacion
0     Jose  Contabilidad                2008
1    Jesus    Ingenieria                2012
2     Gema    Ingenieria                2004
3    Julia          RRHH                2014

df4
   Departamento supervisor
0  Contabilidad     Javier
1    Ingenieria    Daniela
2          RRHH    Carlota

pd.merge(df3, df4)
  Empleado  Departamento  Fecha_contratacion supervisor
0     Jose  Contabilidad                2008     Javier
1    Jesus    Ingenieria                2012    Daniela
2     Gema    Ingenieria                2004    Daniela
3    Julia          RRHH                2014    Carlota


El ``DataFrame`` tiene una nueva columna **supervisor**, dónde la información está repetida en una o muchas localizaciones, dónde coincide el campo **Departamento**.

### Many-to-many joins

Conceptualmente el join Many-to-many o Muchos-a-Muchos es el más complejo, pero es el caso menos común en la práctica. Si las columnas en ambos lados tienen duplicados, el resultado será este tipo de cruce. Vamos con un ejemplo para tenerlo más claro:

Considera que tenemos el ``DataFrame`` dónde se muestra las skills o habilidades de cada Departamento. Si hacemos un cruce many-to-many, podemos saber todas las skills de todos los empleados.

In [22]:
df5 = pd.DataFrame({'Departamento': ['Contabilidad', 'Contabilidad',
                              'Ingenieria', 'Ingenieria', 'RRHH', 'RRHH'],
                    'skills': ['Matematicas', 'Hojas de Cálculo', 'Programación', 'Linux',
                               'Hojas de Cálculo', 'Organización']})
display('df1', 'df5', "pd.merge(df1, df5)")

,Empleado,Departamento
0,Jose,Contabilidad
1,Jesus,Ingenieria
2,Gema,Ingenieria
3,Julia,RRHH
,Departamento,skills
0,Contabilidad,Matematicas
1,Contabilidad,Hojas de Cálculo
2,Ingenieria,Programación
3,Ingenieria,Linux
4,RRHH,Hojas de Cálculo


Estos tres tipos de Joins pueden ser usados con otras herramientas de Pandas para implementar un gran abanico de funcionalidades. **SPOILER**: En la práctica, los Datasets distan de estar tan limpios cómo el que hemos visto. En la sección siguientes veremos algunas opciones que tiene el ``pd.merge()`` para poder *tunear* las operaciones de cruce.   

## Especificación de la clave de cruce

Ya hemos visto el comportamiento que nos da ``pd.merge()`` por defecto. Parece que uno o más nombres de columnas que sean coincidentes de ambos inputs son los que usan para el cruce. Sin embargo, en el mundo real no tenemos nombres iguales en la mayoría de los casos. En este caso, podemos usar las siguientes funcionalidades de ``pd.merge()``.

### La keyword ``on``:

Con esta keywork podremos especificar cuáles son las claves que se van a usar en el cruce. El formato será un nombre de columna o una lista con los diferentes nombres de columna:

In [23]:
display('df1', 'df2', "pd.merge(df1, df2, on='Empleado')")

df1
  Empleado  Departamento
0     Jose  Contabilidad
1    Jesus    Ingenieria
2     Gema    Ingenieria
3    Julia          RRHH

df2
  Empleado  Fecha_contratacion
0     Gema                2004
1     Jose                2008
2    Jesus                2012
3    Julia                2014

pd.merge(df1, df2, on='Empleado')
  Empleado  Departamento  Fecha_contratacion
0     Jose  Contabilidad                2008
1    Jesus    Ingenieria                2012
2     Gema    Ingenieria                2004
3    Julia          RRHH                2014

Esta funcionalidad sólo funcionará si tenemos la columna de Empleado en ambas tablas.

### Las Keywords ``left_on`` y ``right_on``.

En los casos en los que no tengamos los mismos nombres de columna en ambos ``DataFrames``, tendremos que recurrir a estos parámetros para que cómo indica la palabra en inglés, dispongamos los nombres de columnas que van a cada lado.

In [24]:
df3 = pd.DataFrame({'Nombre': ['Jose', 'Jesus', 'Gema', 'Julia'],
                    'Salario': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="Empleado", right_on="Nombre")')

df1
  Empleado  Departamento
0     Jose  Contabilidad
1    Jesus    Ingenieria
2     Gema    Ingenieria
3    Julia          RRHH

df3
  Nombre  Salario
0   Jose    70000
1  Jesus    80000
2   Gema   120000
3  Julia    90000

pd.merge(df1, df3, left_on="Empleado", right_on="Nombre")
  Empleado  Departamento Nombre  Salario
0     Jose  Contabilidad   Jose    70000
1    Jesus    Ingenieria  Jesus    80000
2     Gema    Ingenieria   Gema   120000
3    Julia          RRHH  Julia    90000

Si nos fijamos, ahora tenemos una columna redundante, por lo que podremos usar el método ``drop()`` para eliminar la columna Nombre:

In [25]:
pd.merge(df1, df3, left_on="Empleado", right_on="Nombre").drop('Nombre', axis=1)

,Empleado,Departamento,Salario
0,Jose,Contabilidad,70000
1,Jesus,Ingenieria,80000
2,Gema,Ingenieria,120000
3,Julia,RRHH,90000


### Las Keyword ``left_index`` y ``right_index``:

En ocasiones, en vez de cruzar por una columna, necesitamos cruzar por un índice de tabla. Por ejemplo, imaginemos este caso:

In [26]:
df1a = df1.set_index('Empleado')
df2a = df2.set_index('Empleado')
display('df1a', 'df2a')

,Departamento
Empleado,
Jose,Contabilidad
Jesus,Ingenieria
Gema,Ingenieria
Julia,RRHH
,Fecha_contratacion
Empleado,
Gema,2004
Jose,2008
Jesus,2012


Ahora podemos usar este índice alfanumérico para unir las columnas usando los parámetros de la función ``pd.merge()``, ``left_index`` y/o ``right_index``: 

In [27]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,Departamento
Empleado,
Jose,Contabilidad
Jesus,Ingenieria
Gema,Ingenieria
Julia,RRHH
,Fecha_contratacion
Empleado,
Gema,2004
Jose,2008
Jesus,2012


Para solucionarnos la vida, el objeto ``DataFrame`` tiene el método ``join()`` que realiza un cruce por índice por defecto, veamos un ejemplo:

In [28]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,Departamento
Empleado,
Jose,Contabilidad
Jesus,Ingenieria
Gema,Ingenieria
Julia,RRHH
,Fecha_contratacion
Empleado,
Gema,2004
Jose,2008
Jesus,2012


Si lo que nos interesa es hacer un mix de índices y columnas a la hora de cruzar, podremos usar indistintamente las columnas de ``left_index`` con ``right_on`` o ``left_on`` con ``right_index`` para llegar al mismo resultado:

In [29]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='Nombre')")

df1a
          Departamento
Empleado              
Jose      Contabilidad
Jesus       Ingenieria
Gema        Ingenieria
Julia             RRHH

df3
  Nombre  Salario
0   Jose    70000
1  Jesus    80000
2   Gema   120000
3  Julia    90000

pd.merge(df1a, df3, left_index=True, right_on='Nombre')
   Departamento Nombre  Salario
0  Contabilidad   Jose    70000
1    Ingenieria  Jesus    80000
2    Ingenieria   Gema   120000
3          RRHH  Julia    90000

Todas estas opciones funcionan también con índices múltibles o con múltiples columnas, la interfaz para este caso es igualmente simple. Para más información sobre los cruces, podemos recurrir siempre a esta excelente sección de la documentación oficial: ["Merge, Join, and Concatenate" section](http://pandas.pydata.org/pandas-docs/stable/merging.html)

## Especificar conjuntos aritméticos para los Joins

En los ejemplos anteriores, hemos visto muchas de las consideraciones a la hora de hacer Joins de Pandas. Una que no hemos tocado ha sido el conjunto aritmético usado en un Join. ¿Qué es un conjunto aritmético? Veámoslo con un ejemplo:

In [30]:
df6 = pd.DataFrame({'Nombre': ['Pedro', 'Pablo', 'Maria'],
                    'Comida': ['Pescado', 'Alubias', 'Pan']},
                   columns=['Nombre', 'Comida'])
df7 = pd.DataFrame({'Nombre': ['Maria', 'Jose'],
                    'Bebida': ['Vino', 'Cerveza']},
                   columns=['Nombre', 'Bebida'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
  Nombre   Comida
0  Pedro  Pescado
1  Pablo  Alubias
2  Maria      Pan

df7
  Nombre   Bebida
0  Maria     Vino
1   Jose  Cerveza

pd.merge(df6, df7)
  Nombre Comida Bebida
0  Maria    Pan   Vino

Tenemos dos Datasets que tienen sólo un registro en común según su clave de cruce, María. En las tablas no tenemos el mismo conjunto de variables o features para cada persona, por lo que perdemos información, **tienen diferentes conjuntos aritméticos.** Por defecto, el resultado contiene la intersección de dos sets de inputs. Esto ya lo hemos conocido como *inner join*. Podemos especificar cómo queremos hacer el cruce con la keywork o parámetro ``how=`` que por defecto es ``"inner"``:

In [31]:
pd.merge(df6, df7, how='inner')

,Nombre,Comida,Bebida
0,Maria,Pan,Vino


Las otras opciones serían ``outer``, ``left`` y ``right``. Acordémonos que *outer join* devuelve un cruce sobre la unión del input de columnas, y las rellena en caso no coincidente con NAs:

In [32]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
  Nombre   Comida
0  Pedro  Pescado
1  Pablo  Alubias
2  Maria      Pan

df7
  Nombre   Bebida
0  Maria     Vino
1   Jose  Cerveza

pd.merge(df6, df7, how='outer')
  Nombre   Comida   Bebida
0  Pedro  Pescado      NaN
1  Pablo  Alubias      NaN
2  Maria      Pan     Vino
3   Jose      NaN  Cerveza

El *left join* y el *right join* por su parte, devuelve el cruce sobre las entradas de la izquierda o de la derecha respectivamente. Por ejemplo:

In [33]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
  Nombre   Comida
0  Pedro  Pescado
1  Pablo  Alubias
2  Maria      Pan

df7
  Nombre   Bebida
0  Maria     Vino
1   Jose  Cerveza

pd.merge(df6, df7, how='left')
  Nombre   Comida Bebida
0  Pedro  Pescado    NaN
1  Pablo  Alubias    NaN
2  Maria      Pan   Vino

In [34]:
display('df6', 'df7', "pd.merge(df6, df7, how='right')")

df6
  Nombre   Comida
0  Pedro  Pescado
1  Pablo  Alubias
2  Maria      Pan

df7
  Nombre   Bebida
0  Maria     Vino
1   Jose  Cerveza

pd.merge(df6, df7, how='right')
  Nombre Comida   Bebida
0  Maria    Pan     Vino
1   Jose    NaN  Cerveza

Si os fijáis, el output de filas corresponde con las entradas/filas que teníamos en el input izquierdo. Usando el parámetro ``how='right'`` el funcionamiento es el mismo.

## Solapar los nombres de columna: La Keyword ``suffixes``

Finalmente, puedes acabar en un caso en el que dos inputs o ``DataFrames`` tengas de nombres de columna en conflicto:

In [35]:
df8 = pd.DataFrame({'Nombre': ['Pepe', 'Joaquin', 'Laura', 'Sofia'],
                    'Rango': [1, 2, 3, 4]})
df9 = pd.DataFrame({'Nombre': ['Pepe', 'Joaquin', 'Laura', 'Sofia'],
                    'Rango': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="Nombre")')

df8
    Nombre  Rango
0     Pepe      1
1  Joaquin      2
2    Laura      3
3    Sofia      4

df9
    Nombre  Rango
0     Pepe      3
1  Joaquin      1
2    Laura      4
3    Sofia      2

pd.merge(df8, df9, on="Nombre")
    Nombre  Rango_x  Rango_y
0     Pepe        1        3
1  Joaquin        2        1
2    Laura        3        4
3    Sofia        4        2

Como tenemos un output con dos columnas en conflicto, la función de ``pd.merge()`` es inteligente y pondrá por defento los sufijos de ``_x`` y ``_y`` para hacerlas únicas. Si los resultados son algo *feos* podemos recurrir al parámetro del método ``suffixes``:

In [36]:
display('df8', 'df9', 'pd.merge(df8, df9, on="Nombre", suffixes=["_L", "_R"])')

df8
    Nombre  Rango
0     Pepe      1
1  Joaquin      2
2    Laura      3
3    Sofia      4

df9
    Nombre  Rango
0     Pepe      3
1  Joaquin      1
2    Laura      4
3    Sofia      2

pd.merge(df8, df9, on="Nombre", suffixes=["_L", "_R"])
    Nombre  Rango_L  Rango_R
0     Pepe        1        3
1  Joaquin        2        1
2    Laura        3        4
3    Sofia        4        2

Estos sufijos funcionan con cualquier configuración vista de los Joins, también con múltiples columnas en conflicto.

Para más información sobre estos casos podéis ver también la sección que vimos anteiormente en **Pandas 1 de Agregación y agrupación** donde tenemos más contexto sobre el álgebra relacional. También podemos acudir a esta sección de la documentación oficial de pandas [Pandas "Merge, Join and Concatenate" documentation](http://pandas.pydata.org/pandas-docs/stable/merging.html).

## Vamos con un ejemplo: US States Data

Vamos a realizar operaciones de cruce y unión con diferentes orígenes de datos. Aquí vamos a usar un ejemplo de datos acerca de los estados de (valga la redundancia) de Estados Unidos de América y su población.

In [ ]:
# Podemos usar estan sentencias de shell (Linux) para descargarlo, pero ya lo tenemos. ¿Sabrías hacerlo para Windows?
#!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
#!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
#!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

Primero vamos a usar la función de Pandas ``read_csv()``:

In [ ]:
import pandas as pd
import numpy as np

In [38]:
pop = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv')
areas = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv')
abbrevs = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv')

display('pop', 'areas', 'abbrevs')

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0
...,...,...,...,...
2539,USA,total,2010,309326295.0
2540,USA,under18,2011,73902222.0
2541,USA,total,2011,311582564.0
2542,USA,under18,2012,73708179.0


Teniendo esta información, digamos que queremos computar un resultado sencillo (de primeras): Ordenar por rango los estados y territorios por su densidad de población de 2010. Claramente tenemos los datos necesarios para encontrar el resultado, pero tenemos que, en primer lugar, combinar los tres ``DataFrames``.

Empezaremos haciendo un **Many-to-one** que nos dará el nombre del estado completo con su población. Queremos cruzar basándonos en el campo ``state/region`` columna de la tabla ``pop`` y de ``abbreviation`` (abreviación), columna de la tabla ``abbrevs``. Usaremos el tipo de cruce *Outer* para asegurar que ningún dato se nos pierda en el cruce, tenga o no coincidencia en todas las tablas.

In [41]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', axis = 1) # Nos quitamos los duplicados
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Vamos a asegurarnos de que no tengamos ningúna pérdida de información por aquí, la mejor manera es evaluar si tenemos nulos en las columnas:

In [43]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Ouch! Tenemos nulos en la columna de ``population``, vamos a ver dónde pueden estan:

In [44]:
merged['population'].isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
2539    False
2540    False
2541    False
2542    False
2543    False
Name: population, Length: 2544, dtype: bool

In [45]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


Parece que Puerto Rico no tiene valores de población para algunos años de principios de los 90; probablemente se trata de que no tenemos ese dato en su fuente correspondiente.

Lo más importante es que algunos de estos nuevos estados también pueden estar en Null, ¡lo que significa que tenemos que estar atentos con la key de abreviatura (``abbrevs``)!

In [46]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

Podemos rapidamente inferir el problema que tenemos, nuestros datos de población incluyen entradas de Puerto Rico (PR) y el campo estado parece ser (USA), pero no tenemos esa información en la tabla de abreviaturas para la key que en este caso es PR. Lo que podemos hacer es añadir esta información para los casos que hemos visto que están nulos directamente:

In [47]:
merged.loc[merged['state/region'] == 'PR', 'state']

2448    NaN
2449    NaN
2450    NaN
2451    NaN
2452    NaN
2453    NaN
2454    NaN
2455    NaN
2456    NaN
2457    NaN
2458    NaN
2459    NaN
2460    NaN
2461    NaN
2462    NaN
2463    NaN
2464    NaN
2465    NaN
2466    NaN
2467    NaN
2468    NaN
2469    NaN
2470    NaN
2471    NaN
2472    NaN
2473    NaN
2474    NaN
2475    NaN
2476    NaN
2477    NaN
2478    NaN
2479    NaN
2480    NaN
2481    NaN
2482    NaN
2483    NaN
2484    NaN
2485    NaN
2486    NaN
2487    NaN
2488    NaN
2489    NaN
2490    NaN
2491    NaN
2492    NaN
2493    NaN
2494    NaN
2495    NaN
Name: state, dtype: object

In [48]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

Nos hemos librado de los nulos en la columna de ``state``.

Vamos a seguir un procedimiento similar con los datos de área. Examinando nuestros resultado, querremos unir con también con la columna ``state``:

In [ ]:
merged

In [49]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


De nuevo, vemos si tenemos nulos:

In [50]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

Tenemos algunos nulos en la columna ``area``, vamos a echar un vistazo:

In [51]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

Vemos que nuestro ``DataFrame`` de areas no tiene datos de area correspondientes de los Estados Unidos (United States) como conjunto. Podríamos insertar el valor correcto, usando por ejemplo la suma de todas las areas de todos los estados, pero para este caso (para no complicarnos la vida) vamos a simplemente quitar los valores nulos y con ellos toda la información relativa a los Estados Unidos como conjunto ya que no es tan relevante para el análisis en cuestión.

In [52]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [53]:
final.isnull().any()

state/region     False
ages             False
year             False
population       False
state            False
area (sq. mi)    False
dtype: bool

Ahora que tenemos los datos que necesitamos, vamos a responder a nuestra pregunta, primero vamos a ver la poción de datos correspondiente con el año 2000, y su población total. Vamos usar la función ``query()`` en este caso para hacer esto más rapidamente.

(**¿Es la primera vez que la véis, cierto?**)

En el caso de no tener disponible la función, es posible que necesitemos la librería ``numexpr``.

In [54]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Ahora vamos a computar la densidad de población y mostrarla en orden. **Pero primero lo más importante, reindexar los datos.**

In [58]:
data2010['dessitisieg'] = data2010['population'] / data2010['area (sq. mi)']

C:\Users\Fernando Carrasco\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [59]:
data2010

,state/region,ages,year,population,area (sq. mi),dessitisieg
state,,,,,,
Alabama,AL,total,2010,4785570.0,52423.0,91.287603
Alaska,AK,total,2010,713868.0,656425.0,1.087509
Arizona,AZ,total,2010,6408790.0,114006.0,56.214497
Arkansas,AR,total,2010,2922280.0,53182.0,54.948667
California,CA,total,2010,37333601.0,163707.0,228.051342
Colorado,CO,total,2010,5048196.0,104100.0,48.493718
Connecticut,CT,total,2010,3579210.0,5544.0,645.600649
Delaware,DE,total,2010,899711.0,1954.0,460.445752
District of Columbia,DC,total,2010,605125.0,68.0,8898.897059


In [57]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

KeyError: "None of ['state'] are in the columns"

In [56]:
density

state
Alabama                   91.287603
Alaska                     1.087509
Arizona                   56.214497
Arkansas                  54.948667
California               228.051342
Colorado                  48.493718
Connecticut              645.600649
Delaware                 460.445752
District of Columbia    8898.897059
Florida                  286.597129
Georgia                  163.409902
Hawaii                   124.746707
Idaho                     18.794338
Illinois                 221.687472
Indiana                  178.197831
Iowa                      54.202751
Kansas                    34.745266
Kentucky                 107.586994
Louisiana                 87.676099
Maine                     37.509990
Maryland                 466.445797
Massachusetts            621.815538
Michigan                 102.015794
Minnesota                 61.078373
Mississippi               61.321530
Missouri                  86.015622
Montana                    6.736171
Nebraska              

In [61]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [62]:
final

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
...,...,...,...,...,...,...
2491,PR,under18,2010,896945.0,Puerto Rico,3515.0
2492,PR,under18,2011,869327.0,Puerto Rico,3515.0
2493,PR,total,2011,3686580.0,Puerto Rico,3515.0
2494,PR,under18,2012,841740.0,Puerto Rico,3515.0


El resultado es un ranking de los Estados de US además de Washinton DC y Puerto Rico, en orden, según el censo de población de 2010 y con sus residentes por milla cuadrada.

Vamos a ver los estados que tienen menos densidad de población:

In [63]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

Vemos que el Estado menos poblado (y con diferencia) es Alaska, prácticamente tiene un residente por milla cuadrada (WOW). Este tipo de trabajo con los datos es muy común, espero que este ejemplo con datos del mundo real te haya servido para hacerte una idea de cómo poder combinar las fuentes de datos correctamente para calcular lo necesario para llegar al resultado planteado.